In [12]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA
from functions import lasso_loss, logreg_loss
from scipy.linalg import hadamard

In [13]:
n_samples = 1000
n_test = 100
n = 10
p = 8
model = 'logreg'

loss = {'lasso': lasso_loss, 'logreg': logreg_loss}[model]

H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = np.random.randn(p, n_samples)
D = np.random.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = np.random.randn(n, n_samples)

z_test = np.random.randn(p, n_test)
eps_test = np.random.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if model == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = lbda_max * 0.1
print(X.shape)

(10, 1000)


In [14]:
ista = LISTA(D, lbda, 1000, model=model)
print(loss(np.zeros((p, n_samples)), X, D, lbda))
f_star = loss(ista.transform(X), X, D, lbda)
print(f_star)

6.931471805599452
5.498817604903095


In [15]:
n_layers = 10
lista = LISTA(D, lbda, n_layers, model=model, l_star=f_star)
loss(lista.transform(X), X, D, lbda)

5.530959064404181

In [16]:
lista.fit(X, l_rate=1e-1, max_iter=1000, verbose=True)

it 0, loss = 5.53e+00, grad = 8.26e-02
it 100, loss = 5.52e+00, grad = 1.52e-02
it 200, loss = 5.52e+00, grad = 1.25e-02
it 300, loss = 5.52e+00, grad = 1.20e-02
it 400, loss = 5.52e+00, grad = 1.17e-02
it 500, loss = 5.52e+00, grad = 1.17e-02
it 600, loss = 5.52e+00, grad = 1.16e-02
it 700, loss = 5.52e+00, grad = 1.16e-02
it 800, loss = 5.52e+00, grad = 1.16e-02
it 900, loss = 5.52e+00, grad = 1.16e-02


In [ ]:
lista_one = LISTA(D, lbda, 1, model=model)
lista_one.fit(X, l_rate=1e-2, max_iter=10000, batch_size=1000, verbose=True)

In [ ]:
f_star = loss(LISTA(D, lbda, 1000, model=model).transform(X_test), X_test, D, lbda)
f0 = loss(np.zeros_like(z_test), X_test, D, lbda)

In [ ]:
ista = LISTA(D, lbda, n_layers, model=model)
print('Avg test loss using ISTA with %d iterations: %.2e' %
      (n_layers, (loss(ista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

In [ ]:
print('Avg test loss using LISTA with %d layers: %.2e' %
      (n_layers, (loss(lista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))